# Introduction: OCHA

The Admin dataset from **OCHA** is a collection of administrative boundaries used for humanitarian planning, disaster response, and development efforts. The dataset contains **names**, **codes** and **polygons** for each administrative unit.

There are 2 ways in which you can access OCHA admin data using the rapida tool:
1. using the API
2. Using the (CLI) Commandline Interface

> **💡 Tip:**
Run the code cells by hitting `SHIFT` + `ENTER` on your keyboard

## 1.Using the API

Step 1: Import the necessary packages.

In [1]:
import json
from cbsurge.admin import ocha, save
from cbsurge.admin.ocha import fetch_admin, countries_for_bbox, fetch_ocha_admin_levels, fetch_admin


For the purposes of this section, we are going to use a specific bounding box

`bbox = (33.681335, -0.131836, 35.966492, 1.158979)`

You can check which countries are covered by the specified bounding box by running:
In the output of the cell below, you will see a tuple of the ISO3 codes of countries covered by the specified bounding box

In [2]:
bbox = (33.681335, -0.131836, 35.966492, 1.158979)
countries = countries_for_bbox(bounding_box=bbox)
countries

('KEN', 'UGA')

You can also check which admin levels are available for the above countries by running `fetch_ocha_admin_levels`

In [3]:
admin_levels = fetch_ocha_admin_levels(iso3_country='KEN')
admin_levels

{0: ('Admin0', 0), 1: ('Admin1', 1), 2: ('Admin2', 2)}

To fetch the admin dataset, you can use the `fetch_admin` function as follows:

In [4]:
data = fetch_admin(bbox=bbox, admin_level=1, clip=False) # the fetch_admin returns geojson object
if data is not None:
    save(geojson_dict=data, dst_path='data/admin1_ocha.geojson', layer_name='admin1')

If you open the `data` folder, you will see the `admin1.geojson` file within the folder

## 2. Using the command line interface

You can also fetch admin data using the rapida cli. The following command is necessary:

> **💡 Note:**
Before you run this command, you **have to be authenticated** and have the rapida tool **initialized** the steps to authenticate and initialize are specified in this [notebook](https://rapida.undpgeohub.org/hub/user-redirect/lab/tree/jin/20240213_training/20240213_population.ipynb)


In [5]:
! rapida admin ocha --bbox=33.681335,-0.131836,35.966492,1.158979 --admin_level=2 --clip data/admin2_ocha_cli.geojson

2025-02-13 14:36:10,878 - INFO - HTTP Request: GET https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/World_Countries_(Generalized)/FeatureServer/0/query?where=1%3D1&outFields=*&geometry=33.681335,-0.131836,35.966492,1.158979&geometryType=esriGeometryEnvelope&inSR=4326&spatialRel=esriSpatialRelIntersects&returnGeometry=false&outSR=4326&f=json "HTTP/1.1 200 OK"
2025-02-13 14:36:11,228 - INFO - HTTP Request: GET https://codgis.itos.uga.edu/arcgis/rest/services/COD_External?f=pjson "HTTP/1.1 200 "
2025-02-13 14:36:11,373 - INFO - HTTP Request: GET https://codgis.itos.uga.edu/arcgis/rest/services/COD_External/KEN_pcode/MapServer?f=pjson "HTTP/1.1 200 "
2025-02-13 14:36:11,729 - INFO - HTTP Request: GET https://codgis.itos.uga.edu/arcgis/rest/services/COD_External/KEN_pcode/FeatureServer/2/query?outFields=%2A&geometry=33.681335%2C+-0.131836%2C+35.966492%2C+1.158979&geometryType=esriGeometryEnvelope&inSR=4326&spatialRel=esriSpatialRelIntersects&returnGeometry=true&outSR=4326&f

Running the `rapida admin ocha` command will give you a help message of the command and the different options that you could use as follows

In [6]:
! rapida admin ocha

Usage: rapida admin ocha [OPTIONS] DESTINATION_PATH

  Fetch admin boundaries from OCHA COD

  Retrieves administrative boundaries of a specific level/levels that
  intersect the area covered iby bbox from OCHA COD database hosted on ArcGIS
  server The admin_level argument can be an integer or a dictionary.

  In case an integer is supplied, the best effort is made to retrieve admin
  boundaries from that specific admin level data layer for all countries
  whose extent intersects the bounding box

  The OCA COD database is hosted on an ArcGIS server instance and is organized
  on a per country basis. As a result this function also uses a per country
  approach to fetch features and merge them into one layer in case the
  supplied bounding box covers several countries.

  To save the result as a file, for instance, the following command can be
  executed to extract admin 0 data for Rwanda and Burundi as GeoJSON file:

  rapida admin ocha -b 33.681335,-0.131836,35.966492,1.158979 -l 2 -

## 3. Visualizing the downloaded admin data

We use the [leafmap](https://github.com/opengeos/leafmap) package to visualize the data on this notebook

In [7]:
import leafmap

In [8]:
m = leafmap.Map(center=(0.35, 38.48), zoom=6) # set the center and zoom level to an appropriate values to see the result without having to pan the map

In [9]:
geojson_path = "data/admin2_ocha_cli.geojson"
m.add_geojson(geojson_path, layer_name="admin2")
m